# Plane Fitting and Keyframe Projection

The following script has two main tasks:
1. Fit a plane into the 3D map points which were triangulated by the SLAM system
2. Project the individual key frames onto this plane and blend them into a single large mosaic

In [3]:
%matplotlib ipympl
import pickle
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *

In [4]:
map_points = pickle.load(open("map_points.pkl", "rb"))
kf_visible_map_points = pickle.load(open("kf_visible_map_points.pkl", "rb"))
kf_poses = pickle.load(open("kf_poses.pkl", "rb"))
kf_frames = pickle.load(open("kf_frames.pkl", "rb"))
kf_kp_matched = pickle.load(open("kf_kp_matched.pkl", "rb"))

### Fit a Plane into the map points using a RANSAC scheme

In [5]:
import numpy as np
import numpy.linalg as la

eps = 0.00001

def svd(A):
    u, s, vh = la.svd(A)
    S = np.zeros(A.shape)
    S[:s.shape[0], :s.shape[0]] = np.diag(s)
    return u, S, vh


def fit_plane_LSE(points):
    # points: Nx4 homogeneous 3d points
    # return: 1d array of four elements [a, b, c, d] of
    # ax+by+cz+d = 0
    assert points.shape[0] >= 3 # at least 3 points needed
    U, S, Vt = svd(points)
    null_space = Vt[-1, :]
    return null_space


def get_point_dist(points, plane):
    # return: 1d array of size N (number of points)
    dists = np.abs(points @ plane) / np.sqrt(plane[0]**2 + plane[1]**2 + plane[2]**2)
    return dists


def fit_plane_LSE_RANSAC(points, iters=1000, inlier_thresh=0.05, num_support_points=None, return_outlier_list=False):
    # points: Nx4 homogeneous 3d points
    # num_support_points: If None perform LSE fit with all points, else pick `num_support_points` random points for fitting
    # return: 
    #   plane: 1d array of four elements [a, b, c, d] of ax+by+cz+d = 0
    #   inlier_list: 1d array of size N of inlier points
    max_inlier_num = -1
    max_inlier_list = None
    
    N = points.shape[0]
    assert N >= 3

    for i in range(iters):
        chose_id = np.random.choice(N, 3, replace=False)
        chose_points = points[chose_id, :]
        tmp_plane = fit_plane_LSE(chose_points)
        
        dists = get_point_dist(points, tmp_plane)
        tmp_inlier_list = np.where(dists < inlier_thresh)[0]
        tmp_inliers = points[tmp_inlier_list, :]
        num_inliers = tmp_inliers.shape[0]
        if num_inliers > max_inlier_num:
            max_inlier_num = num_inliers
            max_inlier_list = tmp_inlier_list
        
        #print('iter %d, %d inliers' % (i, max_inlier_num))

    final_points = points[max_inlier_list, :]
    if num_support_points:
        max_support_points = np.min((num_support_points, final_points.shape[0]))
        support_idx = np.random.choice(np.arange(0, final_points.shape[0]), max_support_points, replace=False)
        support_points = final_points[support_idx, :]
    else:
        support_points = final_points
    print(final_points.shape)
    plane = fit_plane_LSE(support_points)
    
    fit_variance = np.var(get_point_dist(final_points, plane))
    print('RANSAC fit variance: %f' % fit_variance)
    print(plane)

    dists = get_point_dist(points, plane)

    select_thresh = inlier_thresh * 1

    inlier_list = np.where(dists < select_thresh)[0]
    if not return_outlier_list:
        return plane, inlier_list
    else:
        outlier_list = np.where(dists >= select_thresh)[0]
        return plane, inlier_list, outlier_list

In [6]:
map_points_h = cv2.convertPointsToHomogeneous(map_points).reshape(-1, 4)

In [7]:
import time

In [8]:
# limiting num_support_points speeds up this operation significantly
t0 = time.perf_counter()
plane, inlier_list = fit_plane_LSE_RANSAC(map_points_h, iters=1000, inlier_thresh=1, num_support_points=3000, return_outlier_list=False)
dt = time.perf_counter() - t0
print(inlier_list.shape, dt)

(29653, 4)
RANSAC fit variance: 0.048014
[ 4.78195791e-04 -9.75356879e-05  4.98333334e-02 -9.98757428e-01]
(29207,) 1.1049468519631773


### Invert plane if normal points in the "wrong" direction

During fitting the plane can be either fitted in a way that the normal points towards the world origin (parameter d > 0) or in the opposite direction (d < 0). The following calculations assume the plane normal to point into the direction of the world origin. So, in case the plane is fitted with d < 0, the plane normal has to be inverted, that is "flipped" by 180° along its own axis.

In [9]:
if plane[3] < 0:
    plane *= -1

In [10]:
print(plane)

[-4.78195791e-04  9.75356879e-05 -4.98333334e-02  9.98757428e-01]


### Create an orthonormal plane base 

In [11]:
def plane_to_hessian(plane):
    """Convert plane to Hessian normal form (n.x + p = 0)"""
    a, b, c, d = plane
    nn = np.sqrt(a**2+b**2+c**2)
    n = np.array([a/nn, b/nn, c/nn])
    p = d/nn
    return n, p

In [12]:
def make_plane_base(plane):
    """Create a orthonormal plane coordinate base.
    
    Args:
    
        plane (`numpy.ndarray`): Shape (4,). Plane coefficients [a, b, c, d] which fullfill
            ax + by + cz + d = 0.
    
    Returns:
    
        base (`numpy.ndarray`) Shape (3, 4). right-handend orthonormal base in which 2D plane points are 
            expressed. Column vectors of the array correspond to the origin point O of the base, the first and 
            second base vector u, v and the third base vector n which is the plane normal.
    """
    # get two points on the plane for x = 0, y = 0 and x = 1, y = 0, respectively
    z1 = -1*plane[3]/plane[2]
    p1 = np.array([0, 0, z1])
    z2 = -1*(plane[0]+plane[3])/plane[2]
    p2 = np.array([1, 0, z2])
    # first plane base vector u
    u = (p2-p1)/np.linalg.norm(p2-p1)
    # third plane base vector := plane normal
    n, _ = plane_to_hessian(plane)
    # second plane base vector v 
    uxn = np.cross(u, n)
    v = uxn/np.linalg.norm(uxn)
    # assemble base as single matrix
    base = np.stack([p1.T, u.T, v.T, n.T], axis=1)
    return base

In [13]:
base = make_plane_base(plane)
print(base)

[[ 0.00000000e+00  9.99953963e-01  1.87796980e-05 -9.59544200e-03]
 [ 0.00000000e+00  0.00000000e+00  9.99998085e-01  1.95714403e-03]
 [ 2.00419551e+01 -9.59546038e-03  1.95705392e-03 -9.99952047e-01]]


### Convert camera poses from world to plane coordinates

In [14]:
def to_twist(R, t):
    """Convert a 3x3 rotation matrix and translation vector (shape (3,))
    into a 6D twist coordinate (shape (6,))."""
    r, _ = cv2.Rodrigues(R)
    twist = np.zeros((6,))
    twist[:3] = r.reshape(3,)
    twist[3:] = t.reshape(3,)
    return twist

def from_twist(twist):
    """Convert a 6D twist coordinate (shape (6,)) into a 3x3 rotation matrix
    and translation vector (shape (3,))."""
    r = twist[:3].reshape(3, 1)
    t = twist[3:].reshape(3, 1)
    R, _ = cv2.Rodrigues(r)
    return R, t

In [15]:
# first compute R_plane, t_plane from given coordinate systems
# see: https://math.stackexchange.com/questions/1125203/finding-rotation-axis-and-angle-to-align-two-3d-vector-bases
def get_plane_pose(plane_O, plane_u, plane_v, plane_n):
    """Compute the plane pose R_plane, t_plane in world coordinates.
    
    Assumes the world coordinate system to have rotation R = I and zero translation.
    
    Args:
        plane_O (`numpy.ndarray`): Shape (3,). Origin of the plane coordinate base.
        
        plane_u (`numpy.ndarray`): Shape (3,). First base vector of the plane coordinate system.
        
        plane_v (`numpy.ndarray`): Shape (3,). Second base vector of the plane coordinate system.
        
        plane_n (`numpy.ndarray`): Shape (3,). Third base vector of the plane coordinate system, 
            corresponds to plane normal.
            
    Returns:
        R_plane (`numpy.ndarray`), t_plane (`numpy.ndarray`): Rotation matrix with shape (3, 3) and 
        translation vector with shape (3,) which describe the pose of the plane coordinate base in
        the world coordinate frame.
    """
    t_plane = plane_O
    world_x = np.array([1, 0, 0])
    world_y = np.array([0, 1, 0])
    world_z = np.array([0, 0, 1])
    R_plane = np.outer(plane_u, world_x) + np.outer(plane_v, world_y) + np.outer(plane_n, world_z)
    return R_plane, t_plane

In [16]:
R_plane, t_plane = get_plane_pose(base[:, 0], base[:, 1], base[:, 2], base[:, 3])
print(R_plane, t_plane)

[[ 9.99953963e-01  1.87796980e-05 -9.59544200e-03]
 [ 0.00000000e+00  9.99998085e-01  1.95714403e-03]
 [-9.59546038e-03  1.95705392e-03 -9.99952047e-01]] [ 0.          0.         20.04195505]


In [17]:
def pose_world_to_plane(R_plane, t_plane, R_pose, t_pose):
    """Map keyframe poses from the world to plane coordinate frame.

    Args:
        R_plane (`numpy.ndarray`): Shape (3, 3). Rotation matrix of plane coordinate system in world coordinate frame.
        t_plane (`numpy.ndarray`): Shape (3,). Translation vector of plane coordinate system in world coordinate frame.
        
        R_pose (`numpy.ndarray`): Shape (3, 3). Rotation matrix of keyframe in world coordinate frame.
        t_pose (`numpy.ndarray`): Shape (3,). Translation vector of keyframe in world coordinate frame.

    Returns:
        R_pose_plane (`numpy.ndarray`), t_pose_plane (`numpy.ndarray`): Rotation matrix with shape (3, 3) and 
        translation vector with shape (3,) of the keyframe in plane coordinate frame.
    """
    t_pose_plane = t_plane.reshape(3,) + R_plane @ t_pose.reshape(3,)
    R_pose_plane = R_plane @ R_pose
    return R_pose_plane, t_pose_plane

In [18]:
# convert all keyframe poses to plane coordinates
kf_poses_plane = []
for pose in kf_poses:
    R_pose, t_pose = from_twist(pose)
    R_kf_plane, t_kf_plane = pose_world_to_plane(R_plane, t_plane, R_pose, t_pose)
    kf_poses_plane.append((R_kf_plane, t_kf_plane))

In [19]:
R_world = np.eye(3)
t_world = np.zeros((3,))

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
plot_basis(ax, R_world, t_world)
plot_basis(ax, R_plane, t_plane)

R_0, t_0 = from_twist(kf_poses[0])
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = from_twist(kf_poses[1])
plot_basis(ax, R_1, t_1.reshape(3,))

R_0, t_0 = kf_poses_plane[0]
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = kf_poses_plane[1]
plot_basis(ax, R_1, t_1.reshape(3,))

ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")
ax.set_xlim([-20, 20])
ax.set_ylim([-20, 40])
ax.set_zlim([0, 40])
ax.set_aspect(1.0)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

FigureCanvasNbAgg()

### Project image corners onto plane

In [20]:
w = 1920
h = 1080
fx = 1184.51770
fy = 1183.63810
cx = 978.30778
cy = 533.85598

In [21]:
# corners of the image
img_points = np.array([[0, 0],
                       [w, 0],
                       [0, h],
                       [w, h]])

In [22]:
def unproject(img_points, fx, fy, cx, cy):
    """Unproject image points with shape (-1, 2) to camera coordinates."""
    camera_points = np.zeros((img_points.shape[0], 3))
    camera_points[:, 0] = (img_points[:, 0]-cx)/fx
    camera_points[:, 1] = (img_points[:, 1]-cy)/fy
    camera_points[:, 2] = 1.0
    return camera_points

In [23]:
camera_points = unproject(img_points, fx, fy, cx, cy)
camera_points  # image corners in camera coordinates

array([[-0.82591234, -0.45102974,  1.        ],
       [ 0.79500055, -0.45102974,  1.        ],
       [-0.82591234,  0.46141132,  1.        ],
       [ 0.79500055,  0.46141132,  1.        ]])

In [59]:
# adapted from: https://github.com/zdzhaoyong/Map2DFusion/blob/master/src/Map2DRender.cpp
warped_frames = [None for _ in range(len(kf_frames))]
warped_masks = [None for _ in range(len(kf_frames))]
length_pixel = 0.01
view_min = np.array([1e6, 1e6])
view_max = np.array([-1e6, -1e6])

sizes = np.zeros((len(kf_frames), 2), dtype=np.int)
corners_world = np.zeros((len(kf_frames), 2))

for idx, (pose, frame) in enumerate(zip(kf_poses_plane, kf_frames)):
    
    frame_okay = True
    cur_view_min = np.array([1e6, 1e6])
    cur_view_max = np.array([-1e6, -1e6])
    
    # generally applicable because the world coordinate base coincides with the first keyframe and plane 
    # points will always have a positive z_world coordinate. Further, we ensure the plane normal to point
    # towards the world origin.
    downlook = np.array([0,0,-1])
    
    # project image corners from camera to plane coordinates
    plane_points = np.zeros((camera_points.shape[0], 2))
    for i, camera_point in enumerate(camera_points):
        axis = pose[0] @ camera_point
        if np.dot(axis, downlook) < 0.4:
            print("Camera axis is deviating too much from 'down' direction. Skipping to next keyframe.")
            frame_okay = False
            break
        axis = pose[1] - axis*(pose[1][-1]/axis[-1])
        plane_points[i, :] = axis[:2]
        
    if not frame_okay:
        continue
    
    # expand viewport of current frame
    for i, plane_point in enumerate(plane_points):
        if plane_point[0] < cur_view_min[0]:
            cur_view_min[0] = plane_point[0]
        if plane_point[1] < cur_view_min[1]:
            cur_view_min[1] = plane_point[1]
        if plane_point[0] > cur_view_max[0]:
            cur_view_max[0] = plane_point[0]
        if plane_point[1] > cur_view_max[1]:
            cur_view_max[1] = plane_point[1]
            
    # expand overall viewport if necessary
    if cur_view_min[0] < view_min[0]:
        view_min[0] = cur_view_min[0]
    if cur_view_min[1] < view_min[1]:
        view_min[1] = cur_view_min[1]
    if cur_view_max[0] > view_max[0]:
        view_max[0] = cur_view_max[0]
    if cur_view_max[1] > view_max[1]:
        view_max[1] = cur_view_max[1]

    corners_world[idx, :] = cur_view_min
    sizes[idx, :] = ((cur_view_max - cur_view_min)/length_pixel)    
    dst_points = (plane_points - cur_view_min)/length_pixel
    
    # find homography between camera and ground plane points
    transmtx = cv2.getPerspectiveTransform(img_points.astype(np.float32), dst_points.astype(np.float32))
    
    mask = np.full(frame.shape[0:2], 255, dtype=np.uint8)
    
    # warp image & mask
    warped_frame = cv2.warpPerspective(frame, transmtx, tuple(sizes[idx, :]), cv2.INTER_CUBIC, cv2.BORDER_REFLECT)
    warped_mask = cv2.warpPerspective(mask, transmtx, tuple(sizes[idx, :]), cv2.INTER_NEAREST)
    warped_frames[idx] = warped_frame
    warped_masks[idx] = warped_mask

In [60]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.imshow(warped_frames[0][:, :, ::-1])
ax2.imshow(warped_masks[0][:, :])
plt.show()

FigureCanvasNbAgg()

In [26]:
R_world = np.eye(3)
t_world = np.zeros((3,))

fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
plot_basis(ax, R_world, t_world)
plot_basis(ax, R_plane, t_plane)

R_0, t_0 = from_twist(kf_poses[0])
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = from_twist(kf_poses[1])
plot_basis(ax, R_1, t_1.reshape(3,))

R_0, t_0 = kf_poses_plane[0]
plot_basis(ax, R_0, t_0.reshape(3,))
R_1, t_1 = kf_poses_plane[1]
plot_basis(ax, R_1, t_1.reshape(3,))

#ax.scatter(points_world[:1000, 0], points_world[:1000, 1], points_world[:1000, 2], s=1, c="red")
#ax.scatter(points_plane[:1000, 0], points_plane[:1000, 1], s=1, c="green")

ax.scatter(camera_points[:, 0], camera_points[:, 1], camera_points[:, 2], c="cyan")
ax.scatter(plane_points[:, 0], plane_points[:, 1], c="magenta")

ax.scatter(base[0, 0], base[1, 0], base[2, 0], c="orange")
ax.scatter(base[0, 0]+base[0, 1], base[1, 0]+base[1, 1], base[2, 0]+base[2, 1], c="red")
ax.scatter(base[0, 0]+base[0, 2], base[1, 0]+base[1, 2], base[2, 0]+base[2, 2], c="green") 
ax.scatter(base[0, 0]+base[0, 3], base[1, 0]+base[1, 3], base[2, 0]+base[2, 3], c="blue")
ax.set_xlim([-20, 20])
ax.set_ylim([-20, 40])
ax.set_zlim([0, 40])
ax.set_aspect(1.0)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

FigureCanvasNbAgg()

### Stitch frames together using OpenCV Blenders

In [27]:
corners_images = (corners_world - view_min)/length_pixel
corners_images = corners_images.astype(np.int)

In [28]:
# define ROI of stitched mosaic
bottom_right = np.max(corners_images, axis=0) + np.array([sizes[np.argmax(corners_images, axis=0)[0], 0], sizes[np.argmax(corners_images, axis=0)[1], 1]])
result_roi = np.array([0, 0, bottom_right[0], bottom_right[1]])
print("result ROI:", result_roi)

result ROI: [   0    0 4727 7847]


In [29]:
# compute number of channels
blend_strength = 5
result_roi_area = (result_roi[2] - result_roi[0]) * (result_roi[3] - result_roi[1])
blend_width = np.sqrt(result_roi_area) * blend_strength/100
print(blend_width)

304.5191660634844


#### Multiband Blender

In [30]:
num_bands = int(np.ceil(np.log(blend_width)/np.log(2)) - 1)
print("Using bands:", num_bands)

blender = cv2.detail_MultiBandBlender(try_gpu=False, num_bands=num_bands)
blender.prepare(result_roi)

for idx, (frame, mask) in enumerate(zip(warped_frames, warped_masks)):
    if frame is not None:
        blender.feed(frame, mask, tuple(corners_images[idx]))
    
result, result_mask = blender.blend(None, None)

Using bands: 8


In [31]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.imshow(result[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [32]:
cv2.imwrite("result.jpg", result)

True

#### Feather Blender 
(result looks better than with multiband blender)

In [33]:
blender = cv2.detail_FeatherBlender(sharpness=1./blend_width)
blender.prepare(result_roi)

for idx, (frame, mask) in enumerate(zip(warped_frames, warped_masks)):
    if frame is not None:
        blender.feed(frame.astype(np.int16), mask, tuple(corners_images[idx]))
    
result, result_mask = blender.blend(None, None)

In [34]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.imshow(result[:, :, ::-1])
plt.show()

FigureCanvasNbAgg()

In [35]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.imshow(result_mask)
plt.show()

FigureCanvasNbAgg()

In [36]:
cv2.imwrite("result_feather.jpg", result)

True

In [ ]:
# TODO:
# - apply weighted image as in Map2DFusion
# - make background white  

In [ ]:
# the followig code contains the weighted image which is to be used as mask for the blending

In [93]:
weight_image = np.full((h, w), 255, dtype=np.uint8)

In [94]:
center = np.array([0.5*w, 0.5*h])
dismaxinv = 1./np.linalg.norm(center)

In [97]:
for i in range(0, h):
    for j in range(0, w):
        
        dis = (i - center[1])**2 + (j - center[0])**2
        dis = 1 - np.sqrt(dis) * dismaxinv
        
        weight_image[i, j] = dis**2 * 254
        if weight_image[i, j] < 1: 
            weight_image[i, j] = 1

In [98]:
weight_image

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [99]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
weight_image = cv2.cvtColor(weight_image, cv2.COLOR_GRAY2BGR)
ax.imshow(weight_image[:, :, ::-1])
plt.show()

/home/lukas/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()